<a href="https://colab.research.google.com/github/saidurga-kanuganti/bdaa/blob/main/icp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
!tar xf spark-3.0.3-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"

In [4]:
!pip install pyspark

In [5]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Big_Data_Application_ICP_2").setMaster("local[*]")
script = SparkContext.getOrCreate(conf=conf)

Importing file that contains required data

In [6]:
from google.colab import files
files.upload()

Saving icp2.txt to icp2 (5).txt


{'icp2.txt': b'As the Labor Day holiday nears, many people are planning travel and get-togethers to see family and friends.Unfortunately, this is occurring at the same time Covid-19 rates are climbing. The rates of new coronavirus infections are higher than they have been since January. Hospitalizations are also at their highest levels since January. In many parts of the United States, both infections and hospitalizations are higher than they were during Labor Day weekend in 2020.How should people think about Covid-19 safety now, compared to last year? Is it safe to see family and friends? What if extended family members want to stay in a house together -- what are some steps they should take to reduce risk? And how does the start of school affect our risk?To help navigate these questions, we spoke with CNN Medical Analyst Dr.Leana Wen. Wen is an emergency physician and visiting professor of health policy and management at the George Washington University Milken Institute School of Pub

Reading the data file

In [7]:
data = script.textFile('icp2.txt')

In [8]:
data.count()

1

In [9]:
data.take(1)

['As the Labor Day holiday nears, many people are planning travel and get-togethers to see family and friends.Unfortunately, this is occurring at the same time Covid-19 rates are climbing. The rates of new coronavirus infections are higher than they have been since January. Hospitalizations are also at their highest levels since January. In many parts of the United States, both infections and hospitalizations are higher than they were during Labor Day weekend in 2020.How should people think about Covid-19 safety now, compared to last year? Is it safe to see family and friends? What if extended family members want to stay in a house together -- what are some steps they should take to reduce risk? And how does the start of school affect our risk?To help navigate these questions, we spoke with CNN Medical Analyst Dr.Leana Wen. Wen is an emergency physician and visiting professor of health policy and management at the George Washington University Milken Institute School of Public Health. S

Slicing the paragraph into words

In [10]:
import string
def Func(sentence):
  sentence = sentence.translate(sentence.maketrans('','',string.punctuation))
  sentence = sentence.strip()
  return sentence


In [11]:
words = data.flatMap(lambda sentence: Func(sentence).split())

In [12]:
words.count()

245

In [13]:
words.take(10)

['As',
 'the',
 'Labor',
 'Day',
 'holiday',
 'nears',
 'many',
 'people',
 'are',
 'planning']

Deleting numbers from the data dile

In [14]:
words_only = words.filter(lambda x: not x.isnumeric())

In [15]:
words_only.count()

244

Change all words in the paragraph into casesensitive i.e., first alphabet in a word to capital alphabets

In [16]:
capital_words = words_only.map(lambda x: x.capitalize())

In [17]:
cap_words = capital_words.map(lambda x: (x[0], x))

In [18]:
cap_words.count()

244

In [19]:
cap_words.take(25)

[('A', 'As'),
 ('T', 'The'),
 ('L', 'Labor'),
 ('D', 'Day'),
 ('H', 'Holiday'),
 ('N', 'Nears'),
 ('M', 'Many'),
 ('P', 'People'),
 ('A', 'Are'),
 ('P', 'Planning'),
 ('T', 'Travel'),
 ('A', 'And'),
 ('G', 'Gettogethers'),
 ('T', 'To'),
 ('S', 'See'),
 ('F', 'Family'),
 ('A', 'And'),
 ('F', 'Friendsunfortunately'),
 ('T', 'This'),
 ('I', 'Is'),
 ('O', 'Occurring'),
 ('A', 'At'),
 ('T', 'The'),
 ('S', 'Same'),
 ('T', 'Time')]

Deleting all duplicates

In [20]:
distinct_words = cap_words.distinct()

In [21]:
distinct_words.count()

150

In [22]:
distinct_words.take(25)

[('A', 'As'),
 ('D', 'Day'),
 ('N', 'Nears'),
 ('M', 'Many'),
 ('P', 'People'),
 ('A', 'Are'),
 ('P', 'Planning'),
 ('T', 'Travel'),
 ('G', 'Gettogethers'),
 ('T', 'To'),
 ('T', 'This'),
 ('I', 'Is'),
 ('S', 'Same'),
 ('T', 'Time'),
 ('R', 'Rates'),
 ('O', 'Of'),
 ('N', 'New'),
 ('H', 'Higher'),
 ('H', 'Have'),
 ('S', 'Since'),
 ('J', 'January'),
 ('H', 'Hospitalizations'),
 ('A', 'Also'),
 ('T', 'Their'),
 ('I', 'In')]

Gropu all words starting with same letter

In [23]:
words_sorted = distinct_words.reduceByKey(lambda x,y:str(x)+ ',' + ' ' + str(y))

In [24]:
words_sorted.take(10)

[('N', 'Nears, New, Now, Navigate'),
 ('S',
  'Same, Since, States, Should, Stay, Steps, Start, Spoke, Shes, Severe, See, Safety, Safe, Some, School, Said'),
 ('R', 'Rates, Risk, Riskto, Report, Reduce, Reason'),
 ('O', 'Of, One, Occurring, Our, Officials'),
 ('J', 'January, Journey'),
 ('W', 'Weekend, What, Want, Wen, Washington, Were, We, With, Well, Who'),
 ('C',
  'Compared, County, Centers, Covid19, Climbing, Coronavirus, Cnn, Control'),
 ('L', 'Last, Labor, Levels, Lifelines, Likely, Los'),
 ('A',
  'As, Are, Also, A, An, Author, Against, And, At, About, Affect, Analyst, Angeles'),
 ('D', 'Day, Does, Doctors, During, Drleana, Different, Disease')]

Sort the group in alphabetical order

In [25]:
words_sorted_byorder = words_sorted.sortByKey(False)

In [26]:
words_sorted_byorder.take(25)

[('Y', 'Year'),
 ('W', 'Weekend, What, Want, Wen, Washington, Were, We, With, Well, Who'),
 ('V', 'Vaccines, Vaccinated, Visiting, Very'),
 ('U', 'Us, United, University, Unvaccinated'),
 ('T',
  'Travel, To, This, Time, Their, Think, Take, Times, The, Than, They, Together, These, That'),
 ('S',
  'Same, Since, States, Should, Stay, Steps, Start, Spoke, Shes, Severe, See, Safety, Safe, Some, School, Said'),
 ('R', 'Rates, Risk, Riskto, Report, Reduce, Reason'),
 ('Q', 'Questions'),
 ('P',
  'People, Planning, Parts, Physician, Protect, Published, Professor, Policy, Public, Prevention'),
 ('O', 'Of, One, Occurring, Our, Officials'),
 ('N', 'Nears, New, Now, Navigate'),
 ('M', 'Many, Members, Management, More, Medical, Milken, Main'),
 ('L', 'Last, Labor, Levels, Lifelines, Likely, Los'),
 ('J', 'January, Journey'),
 ('I', 'Is, In, If, Infections, It, Institute'),
 ('H',
  'Higher, Have, Hospitalizations, How, Health, Hospitalized, Holiday, Highest, House, Help, Healththings'),
 ('G', 'G